# My first graph in tensorflow

In [1]:
import tensorflow as tf

/home/ec2-user/anaconda3/envs/tf+sklearn_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf

In [3]:
import tensorflow as tf
>>> hello = tf.constant('Hello, TensorFlow!')
>>> sess = tf.Session()
>>> print(sess.run(hello))

b'Hello, TensorFlow!'


In [4]:
print('test of the new tensorflow')

test of the new tensorflow


In [5]:
x =tf.Variable(3, name = 'x')
y =tf.Variable(3, name = 'y')
f = x*x*y + y + 2

In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    f.eval()

In [7]:
graph = tf.Graph()

In [8]:
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph


True

In [9]:
f.graph is graph

False

In [10]:
x2.graph is graph

True

In [11]:
tf.reset_default_graph()

In [12]:
import numpy as np
import sklearn as sk
#from.datasets import fetch_california_housing

In [13]:
import sklearn


In [14]:
import scipy


In [15]:
import sklearn